# BioExplorer - CCFv3
![](../bioexplorer_ccfv3_banner.png)

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer, Vector3, MovieMaker
from tqdm import tqdm
import os

url = 'localhost:5000'
be = BioExplorer(url)
mm = MovieMaker(be)
core = be.core_api()

### Load Atlas

In [ ]:
population_name = 'atlas_ccfv3a_averaged'

In [ ]:
olfactory_bulb_ids = [212, 244, 236, 228, 220]

In [ ]:
status = be.reset_scene()
for region_id in olfactory_bulb_ids:
    try:
        atlas_assembly_name = 'Cells %d' % region_id
        be.remove_assembly(atlas_assembly_name)
        atlas_assembly = be.add_assembly(atlas_assembly_name)
        atlas_model = be.add_atlas(
            assembly_name=atlas_assembly_name,
            population_name=population_name,
            load_cells=True, load_meshes=False, cell_radius=5.0,
            region_sql_filter='guid=%d' % region_id,
        )

        if False:
            atlas_assembly_name = 'Mesh %d' % region_id
            be.remove_assembly(atlas_assembly_name)
            atlas_assembly = be.add_assembly(atlas_assembly_name)
            atlas_model = be.add_atlas(
                assembly_name=atlas_assembly_name,
                population_name=population_name,
                load_cells=False, load_meshes=True,
                region_sql_filter='guid=%d' % region_id,
                mesh_scale=Vector3(1, 1, 1)
            )
    except Exception as e:
        print(e)

In [ ]:
model_ids = be.get_model_ids()['ids']
x = 0
for model_id in model_ids:
    transformation={
        'rotation': [0.0, 0.0, 0.0, 1.0],
        'rotation_center': [0.0, 0.0, 0.0],
        'scale': [1.0, 1.0, 1.0],
        'translation': [x, 0.0, 0.0]
    }
    core.update_model(model_id, transformation=transformation)
    x += 2200


In [ ]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_schema = population_name

db_connection_string = 'postgresql+psycopg2://%s:%s@%s:5432/%s' % (db_user, db_password, db_host, db_name)
print('Connection string: ' + db_connection_string + ', schema: ' + db_schema)

engine = create_engine(db_connection_string)
conn = engine.connect()

In [ ]:
region_colors = dict()

def hex_to_rgb(value):
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) / 256.0 for i in range(0, lv, lv // 3))

with Session(engine) as session:
    data = session.execute('select guid, color_hex_triplet from %s.region' % 'atlas_ccfv3a')
    for d in data.all():
        region_colors[int(d[0])] = hex_to_rgb(d[1])

In [ ]:
model_ids = be.get_model_ids()['ids']
for model_id in tqdm(model_ids):
    material_ids = be.get_material_ids(model_id)['ids'][:-1]
    palette = list()
    opacities = list()
    shading_modes = list()
    specular_exponents = list()
    user_params = list()
    refraction_indices = list()
    glossinesses = list()
    reflection_indices = list()
    for material_id in material_ids:
        c = [1, 0, 0]
        m_id = material_id
        if m_id in region_colors:
            c = region_colors[m_id]
        opacities.append(1.0)
        alpha = 1.0
        shading_modes.append(be.shading_mode.ELECTRON_TRANSPARENCY)
        user_params.append(3.0)
        glossinesses.append(1.0)
        specular_exponents.append(50.0)
        reflection_indices.append(0.0)
        refraction_indices.append(0.95)
        palette.append([alpha * c[0], alpha * c[1], alpha * c[2]])
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        shading_modes=shading_modes, user_parameters=user_params,
        opacities=opacities, specular_exponents=specular_exponents,
        reflection_indices=reflection_indices,
        refraction_indices=refraction_indices, glossinesses=glossinesses,
        diffuse_colors=palette, specular_colors=palette)

In [ ]:
background_color = [0.0, 0.0, 0.0]
status = core.set_renderer(
    head_light=True,
    current='advanced', background_color=background_color,
    subsampling=4, max_accum_frames=128)
params = core.AdvancedRendererParams()
params.gi_ray_length = 1e6
params.shadow_intensity = 1.0
params.soft_shadow_strength = 0.5
params.main_exposure = 1.5
params.max_ray_depth = 1
params.epsilon_multiplier = 50.0
params.use_hardware_randomizer = True
status = core.set_renderer_params(params)

In [ ]:
keys = [
    # Olfactory bulb
    {
        'apertureRadius': 0.0,
        'direction': [0.7469370424463216, 0.3606491560448456, 0.5585850345880041],
        'focalDistance': 1000000.0,
        'origin': [-3120.532189038173, 1835.5169097748767, 2165.927051941915],
        'up': [0.3224948767666639, -0.9311843737781622, 0.16997857656370974]
    }
    ,
    # Side view
    {
        'apertureRadius': 0.0,
        'direction': [-1.3331370380301985e-16, -1.2246467991473532e-16, 1.0],
        'focalDistance': 1000000.0,
        'origin': [7236.727651380601, 4482.658523559142, -9145.317280084804],
        'up': [-4.440892098500626e-16, -1.0, -1.2246467991473537e-16]
    }
]
double_keys = list()
for key in keys:
    double_keys.append(key)
    double_keys.append(key)
mm.build_camera_path(double_keys, 50, 25)

In [ ]:
from tqdm import tqdm
import os

output_folder = '/scratch/videos/atlas/%s/olfactory_bulb_explosion/v1' % population_name
os.makedirs(output_folder, exist_ok=True)

k = 4

nb_frames = mm.get_nb_frames()
model_ids = be.get_model_ids()['ids']

mm.set_current_frame(0)
model_ids = be.get_model_ids()['ids']
for model_id in model_ids:
    transformation={
        'rotation': [0.0, 0.0, 0.0, 1.0],
        'rotation_center': [0.0, 0.0, 0.0],
        'scale': [1.0, 1.0, 1.0],
        'translation': [0.0, 0.0, 0.0]
    }
    core.update_model(model_id, transformation=transformation)


explosion_frame = 0
for frame in tqdm(range(nb_frames)):
    mm.set_current_frame(frame)

    if frame >= nb_frames / 2.0:
        x = 0
        for model_id in model_ids:
            transformation={
                'rotation': [0.0, 0.0, 0.0, 1.0],
                'rotation_center': [0.0, 0.0, 0.0],
                'scale': [1.0, 1.0, 1.0],
                'translation': [x, 0.0, 0.0]
            }
            core.update_model(model_id, transformation=transformation)
            x += 600 * float(explosion_frame) / float(nb_frames / 2.0)
            explosion_frame += 1

    mm.create_snapshot(
        renderer='advanced',
        path=output_folder, base_name='%05d' % frame,
        size=[k * 960, k * 540],
        samples_per_pixel=64)


In [ ]:
core.set_camera(
    current='orthographic',
    orientation=[1.0, 0.0, 0.0, 2.83276944882399e-16],
    position=[7576.438343030802, 4265.342891871651, -8852.298404831818],
    target=[7576.438343030802, 4265.342891871657, 5687.5],
)
params = core.OrthographicCameraParams()
params.height = 10000
status = core.set_camera_params(params)

In [ ]:
output_folder = '/scratch/videos/atlas/%s/olfactory_bulb_explosion/v2' % population_name
k = 4
spp = 128
image_size = [k*960, k*270]

In [ ]:
model_ids = be.get_model_ids()['ids']
step = 2200
x = 0
for frame in tqdm(range(100)):
    i = 0
    for model_id in model_ids:
        transformation={
            'rotation': [0.0, 0.0, 0.0, 1.0],
            'rotation_center': [0.0, 0.0, 0.0],
            'scale': [1.0, 1.0, 1.0],
            'translation': [i * x * step, 0.0, 0.0]
        }
        core.update_model(model_id, transformation=transformation)
        i += 1
    mm.create_snapshot(
        renderer='advanced',
        path=output_folder, base_name='%05d' % frame,
        size=image_size, samples_per_pixel=spp)       
    x += 0.01